# Wormhole Throat Geometry Visualization

This notebook visualizes the embedding diagram of the Electro-Lyapunov Microscale Wormhole (ELMW) throat described in [`docs/quantum_microscale_wormhole.md`](quantum_microscale_wormhole.md). It reconstructs the spatial slice governed by the Morris–Thorne metric and highlights how the engineered shape function keeps the throat open at micron scales.

## Metric recap
We work with the standard static, spherically symmetric traversable wormhole metric
\[
ds^2 = -e^{2\Phi(r)} dt^2 + \frac{dr^2}{1 - b(r)/r} + r^2 d\Omega^2,\n\]
where the ELMW model adopts the hybrid shape and redshift functions
\[
b(r) = \frac{a^2}{r} + \alpha\, \frac{r^3}{L^2}, \qquad \Phi(r) = -\beta \frac{(r^2 - a^2)}{L^2}.\n\]
The flare-out condition requires `b(r) < r` for `r > a` so that the embedding diagram remains real. The parameters below follow the microscale design in the accompanying document.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
# Physical parameters (matching Section 2 of the design note)
a = 12e-6   # throat radius (meters)
L = 1e-2    # laboratory length scale (meters)
alpha = 0.02
beta = 0.3

# Shape and redshift functions
def shape_function(r):
    return (a**2) / r + alpha * (r**3) / (L**2)

def redshift_function(r):
    return -beta * (r**2 - a**2) / (L**2)

# Radial grid outside the throat
r_min = 1.001 * a
r_max = 6 * a
r = np.linspace(r_min, r_max, 800)

# Ensure the flare-out condition
valid = shape_function(r) < r
if not np.all(valid):
    raise ValueError('Chosen radial grid violates b(r) < r just outside the throat. Reduce r_max or adjust parameters.')

In [ ]:
def embedding_profile(r_values):
    b = shape_function(r_values)
    denom = r_values - b
    slope_sq = b / denom
    slope = np.sqrt(slope_sq)
    z = np.zeros_like(r_values)
    dr = np.diff(r_values)
    avg_slope = 0.5 * (slope[1:] + slope[:-1])
    z[1:] = np.cumsum(avg_slope * dr)
    return z

z = embedding_profile(r)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
phi = np.linspace(0, 2 * np.pi, 200)
R, Phi = np.meshgrid(r, phi)
Z = np.tile(z, (phi.size, 1))
X = R * np.cos(Phi)
Y = R * np.sin(Phi)

surf = ax.plot_surface(X * 1e6, Y * 1e6, Z * 1e6, cmap=cm.viridis, linewidth=0, antialiased=True, alpha=0.9)
ax.set_xlabel('x [µm]')
ax.set_ylabel('y [µm]')
ax.set_zlabel('z [µm]')
ax.set_title('ELMW wormhole throat embedding surface')
fig.colorbar(surf, shrink=0.6, aspect=10, label='z [µm]')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))
b_vals = shape_function(r)
flare_ratio = b_vals / r
ax.plot(r * 1e6, flare_ratio, label=r'$b(r)/r$')
ax.axhline(1, color='k', linestyle='--', linewidth=1, label='Horizon limit')
ax.set_xlabel('r [µm]')
ax.set_ylabel(r'$b(r)/r$')
ax.set_title('Flare-out diagnostic near the throat')
ax.legend()
plt.show()

## Next steps
- Adjust `alpha` and `beta` to match alternative stabilization regimes and re-run the plots.
- Replace the analytic shape function with numerical solutions of the semiclassical Einstein equations by sampling `T_{\mu\nu}` from simulations.
- Overlay measured throat radius data from NV-center gravimetry once available.